### pandas 6小知识

#### 1、Series与DataFrame的互传
##### 大多时候的计算过程中产生的结果是Series格式，但接下来使用的操作尤其是使用链式语法时，需要衔接着传入DataFrame格式的变量，这时候需要用到互相转换的方法

In [1]:
import pandas as pd

In [2]:
s = pd.Series([0, 1, 2])

# Series转为DataFrame，name参数用于指定转换后的字段名
s = s.to_frame(name='列名')
s

,列名
0,0
1,1
2,2


In [3]:
# 只有单列数据的DataFrame转为Series
s.squeeze()

0    0
1    1
2    2
Name: 列名, dtype: int64

#### 2、随机打乱DataFrame的记录行顺序
##### 有时候我们需要对数据框整体的行顺序进行打乱，譬如在训练机器学习模型时，打乱原始数据顺序后取前若干行作为训练集后若干行作为测试集，这在pandas中可以利用sample()方法快捷实现。
##### sample()方法的本质功能是从原始数据中抽样行记录，默认为不放回抽样，其参数frac用于控制抽样比例，我们将其设置为1则等价于打乱顺序：

In [4]:
df = pd.DataFrame({
    'V1': range(5),
    'V2': range(5)
})
df.sample(frac=1)

,V1,V2
4,4,4
3,3,3
0,0,0
1,1,1
2,2,2


#### 3、利用类别型数据减少内存消耗
##### 当我们的数据框中某些列是由少数几种值大量重复形成时，会消耗大量的内存

In [8]:
import numpy as np
pool = ['A', 'B', 'C', 'D']

# V1列由ABCD大量重复形成
df = pd.DataFrame({
    'V1': np.random.choice(pool, 1000000)
})

# 查看内存使用情况
print(df.memory_usage(deep=True))
print("-" * 10 + "用pandas数据类型中的类别型来极大程度上减少内存消耗" + "-" * 10 )
df['V1'] = df['V1'].astype('category')
df.memory_usage(deep=True)

Index         128
V1       58000000
dtype: int64
----------用pandas数据类型中的类别型来极大程度上减少内存消耗----------


Index        128
V1       1000404
dtype: int64

#### 4、pandas中的object类型陷阱
##### object在pandas中可以代表不确定的数据类型，即类型为object的Series中可以混杂着多种数据类型

In [14]:
s = pd.Series(['111100', '111100', 111100, '111100'])
print(s)
print("-" * 10 + "查看类型分布" + "-" * 10 )
print(s.apply(lambda s: type(s)))
print("-" * 10 + "无法处理的元素会变成缺失值而不报错" + "-" * 10 )
print(s.str.replace('00', '11'))
print("-" * 10 + "先转成对应的类型，再执行" + "-" * 10 )
s.astype('str').str.replace('00', '11')

0    111100
1    111100
2    111100
3    111100
dtype: object
----------查看类型分布----------
0    <class 'str'>
1    <class 'str'>
2    <class 'int'>
3    <class 'str'>
dtype: object
----------无法处理的元素会变成缺失值而不报错----------
0    111111
1    111111
2       NaN
3    111111
dtype: object
----------先转成对应的类型，再执行----------


0    111111
1    111111
2    111111
3    111111
dtype: object

#### 5、快速判断每一列是否有缺失值
##### 在pandas中我们可以对单个Series查看hanans属性来了解其是否包含缺失值，而结合apply()，我们就可以快速查看整个数据框中哪些列含有缺失值

In [16]:
df = pd.DataFrame({
    'V1': [1, 2, None, 4],
    'V2': [1, 2, 3, 4],
    'V3': [None, 1, 2, 3]
})
df.apply(lambda s: s.hasnans)

V1     True
V2    False
V3     True
dtype: bool

#### 6、使用rank()计算排名时的五种策略
##### 可以利用rank()方法计算某一列数据对应的排名信息，但在rank()中有参数method来控制具体的结果计算策略

In [19]:
s = pd.Series([1, 2, 2, 2, 3, 4, 4, 5, 6])
print('-' * 10 + '平均值' + '-' * 10)
print(s.rank(method='average'))
print('-' * 10 + '最小值' + '-' * 10)
print(s.rank(method='min'))
print('-' * 10 + '最大值' + '-' * 10)
print(s.rank(method='max'))
print('-' * 10 + '去重排名，再将每个元素的排名付给相同的每个元素' + '-' * 10)
print(s.rank(method='dense'))
print('-' * 10 + '当多个元素相同时，会根据相同元素再实际Series中的顺序分配排名' + '-' * 10)
s = pd.Series([2, 2, 2, 1, 3])
print(s.rank(method='first'))

----------平均值----------
0    1.0
1    3.0
2    3.0
3    3.0
4    5.0
5    6.5
6    6.5
7    8.0
8    9.0
dtype: float64
----------最小值----------
0    1.0
1    2.0
2    2.0
3    2.0
4    5.0
5    6.0
6    6.0
7    8.0
8    9.0
dtype: float64
----------最大值----------
0    1.0
1    4.0
2    4.0
3    4.0
4    5.0
5    7.0
6    7.0
7    8.0
8    9.0
dtype: float64
----------去重排名，再将每个元素的排名付给相同的每个元素----------
0    1.0
1    2.0
2    2.0
3    2.0
4    3.0
5    4.0
6    4.0
7    5.0
8    6.0
dtype: float64
----------当多个元素相同时，会根据相同元素再实际Series中的顺序分配排名----------
0    2.0
1    3.0
2    4.0
3    1.0
4    5.0
dtype: float64
